#Initial Settings

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')
path='/content/gdrive/My Drive/Colab Notebooks/06_Anomaly/'

import os
os.chdir(path)

!pip install timm
!pip install https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
  Using cached https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip


#Library

In [2]:
import torch.optim as optim
import albumentations as A
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np 
import random
import torch
import timm
import cv2
import os
import time
import sys

from torchsampler.imbalanced import ImbalancedDatasetSampler
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensor
from glob import glob
from tqdm import tqdm
from sklearn.metrics import f1_score

device = torch.device('cuda')

# 학습을 위한 Hyperparameter 셋팅

In [3]:
seed = 51
random.seed(seed)
torch.manual_seed(seed)

lr = 1e-3
folds = 5
batch_size = 16
epochs = 250

resized_image = 700
crop_image = 670

# image augmentation

In [4]:
transform_bottle = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_cable = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_capsule = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_carpet = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_grid = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_hazelnut = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_leather = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_metal_nut = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.RandomRotate90(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_pill = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_screw = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_tile = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_toothbrush = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_transistor = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_wood = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
transform_zipper = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=50, max_h_size=10, max_w_size=10),
    ToTensor()
])

transforms = {"bottle": transform_bottle,
              "cable":transform_cable,
              "capsule":transform_capsule,
              "carpet":transform_carpet,
              "grid": transform_grid,
              "hazelnut":transform_hazelnut,
              "leather":transform_leather,
              "metal_nut":transform_metal_nut,
              "pill":transform_pill,
              "screw":transform_screw,
              "tile":transform_tile,
              "toothbrush": transform_toothbrush,
              "transistor":transform_transistor,
              "wood":transform_wood,
              "zipper":transform_zipper}

transform_pred = A.Compose([
    ToTensor()
])

#Custom Dataset & Model

In [5]:
class Custom_dataset(Dataset):
    def __init__(self, img_arr_list, labels, classes = None, mode='train', transforms=None):
        self.img_arr_list = img_arr_list
        self.labels = labels
        self.classes = classes
        self.labels_arr = np.array(labels)
        self.mode = mode
        self.transforms = transforms
        
    def __len__(self):
        return len(self.img_arr_list)

    def __getitem__(self, idx):
        img = self.img_arr_list[idx]
        
        if self.mode == 'train':
            clas = self.classes[idx]
            img = self.transforms[clas](image=img)['image']

        elif self.mode == 'test':
            img = self.transforms(image=img)['image']

        label = self.labels[idx]

        return img, label

    def get_labels(self):
        return self.labels

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [6]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (resized_image, resized_image))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return img

In [7]:
train_png = sorted(glob('train/*.png'))

train_y = pd.read_csv("open/train_df.csv")
train_labels_str = train_y["label"]
train_classes = train_y["class"]

label_unique = sorted(np.unique(train_labels_str))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels_str]
train_imgs = [img_load(m) for m in tqdm(train_png)]

train_dataset = Custom_dataset(train_imgs, train_labels,classes=train_classes, mode='train',transforms=transforms)

100%|██████████| 4277/4277 [02:26<00:00, 29.25it/s]


# f1 Score function


In [8]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [9]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1, weight = None):
        """if smoothing == 0, it's one-hot method
           if 0 < smoothing < 1, it's smooth method
        """
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.weight = weight
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        assert 0 <= self.smoothing < 1
        pred = pred.log_softmax(dim=self.dim)

        if self.weight is not None:
            pred = pred * self.weight.unsqueeze(0)   

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

# Train
* Augmentation을 class에 따라 다르게 적용하였으나 눈에 띄는 성능향상은 없었음
* 실험이 부족

In [10]:
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

for fold, (train_idx, valid_idx) in enumerate(kfold.split(train_dataset, train_labels)):
    sub_train_imgs = [train_imgs[i] for i in train_idx]
    sub_train_labs = [train_labels[i] for i in train_idx]
    sub_train_clas = [train_classes[i] for i in train_idx]
    sub_train_dataset = Custom_dataset(sub_train_imgs, sub_train_labs, classes= sub_train_clas, mode='train', transforms=transforms)

    sub_valid_imgs = [train_imgs[i] for i in valid_idx]
    sub_valid_labs = [train_labels[i] for i in valid_idx]
    sub_valid_class = [train_classes[i] for i in valid_idx]
    valid_dataset = Custom_dataset(sub_valid_imgs, sub_valid_labs,classes = sub_valid_class, mode='test',transforms=transform_pred)

    train_subsampler = ImbalancedDatasetSampler(sub_train_dataset)
    train_loader = DataLoader(sub_train_dataset, batch_size=batch_size, sampler = train_subsampler)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

    model = Network().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,30,60], gamma=0.5)
    
    # class weight 
    total_lbls = list(set(sub_train_labs))
    lbl_cnt = [sub_train_labs.count(label) for label in total_lbls]
    norm_weights = [1 - (cnt / sum(lbl_cnt)) for cnt in lbl_cnt]
    norm_weights = torch.FloatTensor(norm_weights).cuda()
    criterion = LabelSmoothingLoss(classes=88, smoothing=0.1, weight=norm_weights)

    scaler = torch.cuda.amp.GradScaler() 

    val_loss_plot, val_score_plot = [], []
    global_step = 0
    for epoch in range(epochs):
        start=time.time()
        train_loss = 0
        train_pred=[]
        train_y=[]
        model.train()
        for step, batch in enumerate(train_loader):
            optimizer.zero_grad()
            x = torch.tensor(batch[0], dtype=torch.float32).cuda()
            y = torch.tensor(batch[1], dtype=torch.long).cuda()

            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred, y)
    
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()/len(train_loader)
            train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            train_y += y.detach().cpu().numpy().tolist()

            global_step += 1
        scheduler.step()

        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')

        # Validation
        valid_loss = 0
        valid_pred=[]
        valid_y=[]
        model.eval()

        for batch in (valid_loader):
            optimizer.zero_grad()
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.no_grad():
                pred = model(x)
            loss = criterion(pred, y)

            valid_loss += loss.item()/len(train_loader)
            valid_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            valid_y += y.detach().cpu().numpy().tolist()

        valid_f1 = score_function(valid_y, valid_pred)

        TIME = time.time() - start
        print(f'Valid    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}')
        print(f'time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        val_score_plot.append(valid_f1)
        val_loss_plot.append(valid_loss)

        ##save model
        if np.max(val_score_plot) == val_score_plot[-1]:
            torch.save(model.state_dict(), ".model3/"+str(fold)+".pt")

RuntimeError: ignored